In [1]:
!pip install huggingface huggingface_hub transformers

In [2]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 24.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.1 MB/s eta 0:00:00:00:0100:01
  Attempting u

In [3]:
!pip install datasets accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 5.3 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.5.1
    Uninstalling fsspec-2025.5.1:
      Successfully uninstalled fsspec-2025.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
bigframes 2.8.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.31.0, but you have google-cloud-bigquery 3.25.0 which is incompatible.
bigframes 2.8.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.


In [4]:
!pip install peft

In [5]:
!pip install torchao torchtune

In [6]:
!pip install nltk

In [7]:
!pip install datasets

In [8]:
!pip install evaluate bert-score rouge_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.3 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=e73ab343666df3128d04cb94670c4675de6339d3773e5ba6c57034095bf4281e
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [9]:
from huggingface_hub import login
login()

In [10]:
import torch
import datasets
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
from bert_score import score
import evaluate
from tqdm import tqdm

# Configuration
BASE_MODEL = "meta-llama/Llama-3.2-3B"
ADAPTER_PATH = "D1zzYzz/unsloth-qlora-llama3-3b-QnA-r16"
BATCH_SIZE = 32

# Load evaluation metrics
print("Loading evaluation metrics...")
bleu_metric = evaluate.load("bleu")
rouge_metric = evaluate.load("rouge")
print("Metrics loaded!")

# Setup model with quantization
print(f"Loading model: {BASE_MODEL}")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = 'left'

# Load base model and adapter
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)

model = PeftModel.from_pretrained(base_model, ADAPTER_PATH)
model.eval()
# PERFORMANCE: Compile the model. This adds a one-time overhead for the first
# batch but makes all subsequent batches significantly faster.
print("Compiling the model with torch.compile()...")
model = torch.compile(model)
print("Model loaded and compiled!")

2025-08-03 18:50:48.176388: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754247048.390633      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754247048.455443      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Loading evaluation metrics...


Metrics loaded!
Loading model: meta-llama/Llama-3.2-3B


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/810 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/36.7M [00:00<?, ?B/s]

Compiling the model with torch.compile()...
Model loaded and compiled!


In [11]:
# Load dataset
NUM_TEST_SAMPLES = 1000  # Change this value to adjust the number of validation samples (max 5201)
print("Loading dataset...")
dataset = datasets.load_dataset("databricks/databricks-dolly-15k", split="train")
test_data = dataset.train_test_split(test_size=0.1, seed=42)['test']

# Select subset of test samples if needed
if NUM_TEST_SAMPLES < len(test_data):
    test_data = test_data.select(range(NUM_TEST_SAMPLES))

print(f"Using {len(test_data)} test samples")

# Generate predictions
predictions = []
references = []

print("Generating predictions...")
for i in tqdm(range(0, len(test_data), BATCH_SIZE)):
    # Extract a batch of data from the test set.
    batch = test_data[i:i + BATCH_SIZE]
    
    prompts = []
    # Iterate over each example in the batch to create a formatted prompt.
    # Note: We are using 'context' as the key for the input, to match your tokenize function.
    for instruction, context in zip(batch['instruction'], batch['context']):
        instr = instruction.strip()
        inp = context.strip()
        
        # This prompt structure must exactly match the format the model was trained on.
        # For inference, we provide the instruction and input, and end with "### Response:"
        # to cue the model to start generating its answer.
        if inp:
            # Format for instructions that have an accompanying input/context.
            prompt = f"### Instruction:\n{instr}\n\n### Input:\n{inp}\n\n### Response:\n"
        else:
            # Format for instructions that do not have an input/context.
            prompt = f"### Instruction:\n{instr}\n\n### Response:\n"
        
        prompts.append(prompt)
    
    # Tokenize the batch of prompts.
    # 'padding=True' ensures all sequences in the batch are the same length.
    # 'return_tensors='pt'' returns PyTorch tensors.
    inputs = tokenizer(
            prompts, 
            return_tensors='pt', 
            padding=True, 
            truncation=True, 
            max_length=512, # Ensure this max_length is appropriate for your model.
    )
    
    # Move the tokenized inputs to the same device as the model (e.g., GPU).
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    # Tokenize and generate
    inputs = tokenizer(
            prompts, 
            return_tensors='pt', 
            padding=True, 
            truncation=True, 
            max_length=512, # Use same as training
        )
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    # 1. Use this if we want more real world outputs (stochastic sampling). Output text will never be same.
    # with torch.cuda.amp.autocast():
    #     outputs = model.generate(
    #         **inputs,
    #         max_new_tokens=128,
    #         do_sample=True,
    #         temperature=0.7,
    #         top_p=0.9,
    #         pad_token_id=tokenizer.pad_token_id
    #     )

    # 2. Use this to compare other models and method (Greedy approach). Output text will be same always.
    with torch.cuda.amp.autocast():
        outputs = model.generate(
            **inputs,
            max_new_tokens=1024,
            num_beams=1,  # Use greedy search
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id
        )

    # Decode predictions
    for j, output in enumerate(outputs):
        # Remove input prompt from output
        input_length = inputs['input_ids'][j].shape[0]
        pred_tokens = output[input_length:]
        pred_text = tokenizer.decode(pred_tokens, skip_special_tokens=True)
        predictions.append(pred_text.strip())
    
    references.extend(batch['output'])

print(f"Generated {len(predictions)} predictions")

Loading dataset...


README.md: 0.00B [00:00, ?B/s]

databricks-dolly-15k.jsonl:   0%|          | 0.00/13.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/15011 [00:00<?, ? examples/s]

Using 1000 test samples
Generating predictions...


  0%|          | 0/32 [00:00<?, ?it/s]/tmp/ipykernel_36/1272292320.py:77: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
  0%|          | 0/32 [09:05<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 260.00 MiB. GPU 1 has a total capacity of 14.74 GiB of which 226.12 MiB is free. Process 2873 has 14.52 GiB memory in use. Of the allocated memory 10.64 GiB is allocated by PyTorch, and 3.75 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# Calculate metrics
print("\nCalculating metrics...")

# BLEU Score
try:
    bleu_results = bleu_metric.compute(predictions=predictions, references=[[ref] for ref in references])
    bleu_score = bleu_results['bleu']
    print(f"BLEU: {bleu_score:.4f}")
except Exception as e:
    print(f"BLEU calculation failed: {e}")
    bleu_score = 0.0

# ROUGE Scores
try:
    rouge_results = rouge_metric.compute(predictions=predictions, references=references)
    rouge1 = rouge_results['rouge1']
    rouge2 = rouge_results['rouge2']
    rougeL = rouge_results['rougeL']
    print(f"ROUGE-1: {rouge1:.4f}")
    print(f"ROUGE-2: {rouge2:.4f}")
    print(f"ROUGE-L: {rougeL:.4f}")
except Exception as e:
    print(f"ROUGE calculation failed: {e}")
    rouge1 = rouge2 = rougeL = 0.0


In [ ]:
!pip install -q bert_score

import evaluate

bertscore_metric = evaluate.load("bertscore")
bert_results = bertscore_metric.compute(predictions=predictions, references=references, lang="en")

# We get an average over all the samples
avg_precision = sum(bert_results['precision']) / len(bert_results['precision'])
avg_recall = sum(bert_results['recall']) / len(bert_results['recall'])
avg_f1 = sum(bert_results['f1']) / len(bert_results['f1'])


In [ ]:
# Summary
print("\n" + "="*50)
print("EVALUATION RESULTS")
print("="*50)
print(f"Samples: {len(predictions)}")
print(f"BLEU: {bleu_score:.4f}")
print(f"ROUGE-1: {rouge1:.4f}")
print(f"ROUGE-2: {rouge2:.4f}")
print(f"ROUGE-L: {rougeL:.4f}")
print("\n--- BERT-Score ---")
print(f"Average Precision: {avg_precision:.4f}")
print(f"Average Recall:    {avg_recall:.4f}")
print(f"Average F1 Score:  {avg_f1:.4f}")


In [ ]:
import random

print("\n" + "="*50)
print("SAMPLE PREDICTIONS")
print("="*50)

# Choose 5 random indices
num_samples = min(5, len(predictions))
sample_indices = random.sample(range(len(predictions)), num_samples)

for idx in sample_indices:
    print(f"\nSample {idx + 1}:")
    print(f"Instruction: {test_data['instruction'][idx].strip()}")
    
    input_text = test_data['input'][idx].strip()
    if input_text:
        print(f"Input: {input_text}")
    
    print(f"Reference: {references[idx][:100]}...")
    print(f"Prediction: {predictions[idx][:100]}...")
    print("-" * 30)
